# Dusty wave

## Imports

In [ ]:
import pathlib

import numpy as np
import pandas as pd
import plonk

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot, row
from bokeh.palettes import inferno, viridis, Category10
from bokeh.plotting import figure
output_notebook()

## Exact solution

In [ ]:
"""Exact solutions to dusty wave problem.

The solutions give the magnitude of the perturbations. From
Benítez-Llambay et al. (2019).
"""

import numpy as np

L = 1.0
k = 2.0 * np.pi / L
c_s = 1.0

drho_g = 1.0
rho0_g = 1.0

omega_s = k * c_s

omega = dict()
tstop = dict()

omega[2] = 1.915896 - 4.410541j
tstop[2] = (0.4,)

omega[5] = 0.912414 - 5.493800j
tstop[5] = (0.1, 0.215443, 0.464159, 1.0)


def rho_g(time, omega):
    """Normalized gas density perturbation."""
    const = 1
    return (const * np.exp(-omega * time)).real


def v_g(time, omega):
    """Normalized gas velocity perturbation."""
    const = -1j * omega / omega_s * drho_g / rho0_g
    return (const * np.exp(-omega * time)).real


def rho_d(time, omega, tstop):
    """Normalized dust density perturbation."""
    const = 1 / (1 - omega * tstop) * drho_g / rho0_g
    return (const * np.exp(-omega * time)).real


def v_d(time, omega, tstop):
    """Normalized dust velocity perturbation."""
    const = -1j * omega / omega_s / (1 - omega * tstop) * drho_g / rho0_g
    return (const * np.exp(-omega * time)).real


## Show an example simulation

In [ ]:
def show_example():
    # Run
    run = 'two species'

    # Select snaps to plot
    snap_slice = slice(None, None, 10)

    # Select particles to plot
    particle_slice = slice(None, None, 100)

    # Load data
    directory = pathlib.Path(f'~/runs/multigrain/dustywave/{run}').expanduser()
    sim = plonk.load_sim(prefix='dustywave', directory=directory)
    snaps = sim.snaps[snap_slice]
    
    initial_snap = sim.snaps[0]
    initial_subsnaps = [initial_snap['gas']] + initial_snap['dust']

    for idx, initial_subsnap in enumerate(initial_subsnaps):

        # Plot range
        x_range = (-0.5, 0.5)
        vx_max = np.abs(initial_snap['velocity_x']).max() * 1.05
        vx_range = (-vx_max, vx_max)
        rho_mean = initial_subsnap['density'].mean()
        rho_var = 1.5e-4
        rho_range = (rho_mean * (1 - rho_var), rho_mean * (1 + rho_var))

        # Generate figures
        fig1 = figure(x_range=x_range, y_range=vx_range, plot_width=400, plot_height=400)
        fig2 = figure(x_range=x_range, y_range=rho_range, plot_width=400, plot_height=400)

        for snap, color in zip(snaps, inferno(len(snaps))):
            subsnaps = [snap['gas']] + snap['dust']
            subsnap = subsnaps[idx]
            fig1.scatter(
                subsnap['x'][particle_slice],
                subsnap['velocity_x'][particle_slice],
                line_color=color,
                fill_color=color,
                size=5
            )
            fig2.scatter(
                subsnap['x'][particle_slice],
                subsnap['density'][particle_slice],
                line_color=color,
                fill_color=color,
                size=5
            )

        show(row(fig1, fig2))
        
show_example()

## Do the analysis

Path to data....

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustywave').expanduser()
paths = sorted(list(root_directory.glob('*')))

Calculate $\rho$ and $v_x$ at $x=0$ for each snapshot in each simulation. Each simulation has a data frame.

In [ ]:
A = 1e-4
c_s = 1
npartx = 128

x_boxwidth = 1.0
x_value = -0.5
dx = x_boxwidth / npartx * 1.001

(1.0, 0.1, 0.233333, 0.366667, 0.5)
    
def do_analysis(sim):
    
    n_snaps = len(sim.snaps)
    n_species = len(sim.properties["grain_size"]) + 1

    time = sim.properties['time'].magnitude
    rho = np.zeros((n_snaps, n_species))
    vx = np.zeros((n_snaps, n_species))

    rho_0 = np.zeros(n_species)
    snap = sim.snaps[0]
    for dust_type in range(n_species):
        subsnap = snap[snap['sub_type'] == dust_type]
        rho_0[dust_type] = subsnap['rho'].mean()
        
    for idx, snap in enumerate(sim.snaps):
        position_mask = (snap['x'] < x_value + dx) | (snap['x'] > x_value + x_boxwidth - dx)
        for dust_type in range(n_species):
            species_mask = snap['sub_type'] == dust_type
#            rho_0 = snap[species_mask]['rho'].mean()
            subsnap = snap[position_mask & species_mask]
            rho[idx, dust_type] = (subsnap["rho"].mean() - rho_0[dust_type]) / (A * rho_0[dust_type])
            vx[idx, dust_type] = subsnap["velocity_x"].mean() / (A * c_s)

    arrays = np.hstack((time[:, np.newaxis], rho, vx))
    columns = (
        ["time"]
        + [f"rho.{idx}" for idx in range(n_species)]
        + [f"vx.{idx}" for idx in range(n_species)]
    )
    
    return pd.DataFrame(arrays, columns=columns)

Loop over each simulation.

In [ ]:
dataframes = dict()

for p in paths:
    sim = plonk.load_sim(prefix="dustywave", directory=p)
    name = "-".join(sim.paths['directory'].name.split("=")[-1].split("-")[::-1])
    print(f"Analysis for {name}")

    dataframes[name] = do_analysis(sim)
    del sim

## Plot results

Plot the gas density and velocity at $x=0$ for each run.

In [ ]:
def plot_gas():

    x_range = 0.0, 2.0
    y_range = -1, 1

    p1 = figure(
        title="Normalized gas density perturbation at x=0",
        x_range=x_range,
        y_range=y_range,
        plot_width=400,
        plot_height=300,
    )
    p2 = figure(
        title="Normalized gas velocity perturbation at x=0",
        x_range=x_range,
        y_range=y_range,
        plot_width=400,
        plot_height=300,
    )

    colors = viridis(len(dataframes))

    for (name, df), color in zip(dataframes.items(), colors):

        x = np.array(df["time"])
        y = np.array(df["rho.0"])
        p1.circle(x, y, legend_label=name, line_color=color, fill_color=None)

        y = np.array(df["vx.0"])
        p2.circle(x, y, legend_label=name, line_color=color, fill_color=None)

    show(row(p1, p2))
    
plot_gas()

Plot each species density and velocity.

In [ ]:
t_range = 0.0, 2.0
rho_range = -1, 1
vx_range = -1, 1

ps1 = list()
ps2 = list()

colors = Category10[5]

for run, df in dataframes.items():

    n_species = len([col for col in df.columns if col.startswith('rho')])
    
    p1 = figure(
        title=f"Normalized density perturbation at x=0 for {run}",
        x_range=t_range,
        y_range=rho_range,
        plot_width=400,
        plot_height=300,
    )
    p2 = figure(
        title=f"Normalized velocity perturbation at x=0 for {run}",
        x_range=t_range,
        y_range=vx_range,
        plot_width=400,
        plot_height=300,
    )

    x = np.array(df["time"])
    

    for idx in range(n_species):
        legend_label = f"dust {idx}" if idx != 0 else "gas"
        
        y_numerical = np.array(df[f"rho.{idx}"])
        if idx == 0:
            y_exact = rho_g(time=x, omega=omega[n_species])
        else:
            y_exact = rho_d(time=x, omega=omega[n_species], tstop=tstop[n_species][idx-1])      

        p1.line(x, y_exact, line_color=colors[idx])
        p1.circle(x, y_numerical, line_color=colors[idx], fill_color=None)  # , legend_label=legend_label
        
        y_numerical = np.array(df[f"vx.{idx}"])
        if idx == 0:
            y_exact = v_g(time=x, omega=omega[n_species])
        else:
            y_exact = v_d(time=x, omega=omega[n_species], tstop=tstop[n_species][idx-1]) 

        p2.line(x, y_exact, line_color=colors[idx])
        p2.circle(x, y_numerical, line_color=colors[idx], fill_color=None)  # , legend_label=legend_label

    ps1.append(p1)
    ps2.append(p2)

grid = gridplot([ps2[1], ps2[0], ps1[1], ps1[0]] , ncols=2)
show(grid)